In [27]:
print(f"Lookup ES Aliase Mapping")

Lookup ES Aliase Mapping


In [28]:
!pip install elasticsearch==7.13.0

In [29]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
import jsondiff
import logging
from dotenv import load_dotenv
import socket
import requests
import warnings
warnings.filterwarnings("ignore")

In [30]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [31]:
def get_headers():
    ''' Elasticsearch Header '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')),
            # 'Connection': 'close'
    }

In [32]:
source_es_host = "http://localhost:9201"

In [33]:
es_client = Elasticsearch(hosts="{}".format(source_es_host), headers=get_headers(), timeout=5,  verify_certs=False)
print(es_client.cat.health())
# print(json.dumps(es_client.cat.indices(params={"format": "json"}), indent=2))
print(json.dumps(es_client.cluster.health(), indent=2))

1740178625 22:57:05 docker-cluster yellow 1 1 117 117 0 0 117 0 - 50.0%

{
  "cluster_name": "docker-cluster",
  "status": "yellow",
  "timed_out": false,
  "number_of_nodes": 1,
  "number_of_data_nodes": 1,
  "active_primary_shards": 117,
  "active_shards": 117,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 117,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 50.0
}


In [34]:
def indices_size_list_to_dict(idx):
    ''' transform list to simple dict '''
    ''' 
     {
        "health": "green",
        "status": "open",
        "index": "demo1",
        "uuid": "doRs5jU-SJm4ZRIqTV2duQ",
        "pri": "5",
        "rep": "1",
        "docs.count": "1",
        "docs.deleted": "0",
        "store.size": "8.9kb",
        "pri.store.size": "4.4kb"
      },
  '''
    indices_size_dict = {element.get("index") : element.get("store.size") for element in idx}
    return indices_size_dict

In [35]:
indices_size_list = es_client.cat.indices(params={"format": "json"})
indices_size_dict = indices_size_list_to_dict(indices_size_list)

In [36]:
# print(json.dumps(indices_size_dict, indent=2))

In [37]:
import warnings
import pytest
warnings.filterwarnings('ignore')

In [38]:
from datetime import datetime

In [39]:
''' https://elasticsearch-py.readthedocs.io/en/5.5.1/ '''
doc = {
    "author": "kimchy",
    "text": "Elasticsearch: cool. bonsai cool.",
    "timestamp": datetime.now(),
}
# resp = es_client.index(index="test-index", id=1, body=doc)
resp = es_client.index(index="test-index", id=1, doc_type="test", body=doc)
print(json.dumps(resp["result"], indent=2))

''' http://localhost:9202/test-index/_doc/1 '''
resp = es_client.get(index="test-index", id=1, doc_type="test")
print(resp["_source"])

es_client.indices.refresh(index="test-index")

"updated"
{'author': 'kimchy', 'text': 'Elasticsearch: cool. bonsai cool.', 'timestamp': '2025-02-21T16:57:05.355530'}


{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}

In [40]:
#!pip install ipytest

In [41]:
def get_es_api_alias(es_client):
    ''' get all alias and set to dict'''
    ''' https://localhost:9201/_cat/aliases?format=json '''
    get_alias_old_cluster = es_client.indices.get_alias()
    # print(f"Get alias from old cluster : {json.dumps(get_alias_old_cluster, indent=2)}")

    reset_alias_dict = {}
    for k in get_alias_old_cluster.keys():
        if get_alias_old_cluster.get(k).get('aliases'):
            each_alias = list(get_alias_old_cluster.get(k).get('aliases').keys())
            # print(each_alias)
            reset_alias_dict.update({k : each_alias})
    # print(json.dumps(reset_alias_dict, indent=2))
    # print(f"get_es_api_alias : {reset_alias_dict}")

    return reset_alias_dict

In [42]:
def lookup_old_indices_not_mapped_aliase(es_client):
    ''' Lookup old indices which are not mapped to any aliases excluding indices starting with logstash*, es_*, archieve_* or .'''
    source_idx_lists = list(es_client.indices.get("*"))
    # print(f"ssource_idx_lists : {source_idx_lists}")

    es_aliases_dict = get_es_api_alias(es_client)
    # print(f"es_aliases_dict : {json.dumps(es_aliases_dict, indent=2)}")
    print(f"len(es_aliases_dict for the mapped ES indices) : {len(es_aliases_dict)}")

    # mapped_aliase_list = [",".join(v) for k, v in es_aliases_dict.items() if isinstance(v, (list, str))]
    mapped_aliase_list = [k for k, v in es_aliases_dict.items()]
    # print(json.dumps(mapped_aliase_list, indent=2))

    ''' Lookup old indices which are not mapped to any aliases excluding indices starting with logstash*, es_*, archieve_* or .'''
    unmapped_index_aliase = [
                            index_name for index_name in source_idx_lists 
                             if index_name not in mapped_aliase_list 
                             and not index_name.startswith(".") 
                             and not index_name.startswith("logstash")
                             and not index_name.startswith("es_")
                             and not index_name.startswith("archive_")
                            ]
    # print(json.dumps(unmapped_index_aliase, indent=2))

    query = {
        # "_source": False,
	    'query': {
    	    'match_all': {}
        }
    }
    ''' Get total count for the unmapped index '''
    api_response = {}
    unmapped_index = {}
    api_response.update({"source_es_cluster" : str(es_client)})
    for index_name in unmapped_index_aliase:
        # print(index_name)
        es_count_source = es_client.count(index=index_name, body=query)["count"]
        unmapped_index.update({index_name : {'doc_count' : es_count_source, "store.size" : indices_size_dict.get(index_name)}})
    api_response.update({'unmapped_aliase' : unmapped_index})
                
    return api_response

In [43]:
api_response = lookup_old_indices_not_mapped_aliase(es_client)

len(es_aliases_dict for the mapped ES indices) : 1


In [44]:
print(json.dumps(api_response, indent=2))

{
  "source_es_cluster": "<Elasticsearch([{'host': 'localhost', 'port': 9201}])>",
  "unmapped_aliase": {
    "my-index-01": {
      "doc_count": 1,
      "store.size": "4.2kb"
    },
    "test-index": {
      "doc_count": 1,
      "store.size": "5.3kb"
    },
    "test-index1": {
      "doc_count": 0,
      "store.size": "955b"
    }
  }
}


In [45]:
docs, size = [], []
for idx in list(api_response['unmapped_aliase'].keys()):
    docs.append(api_response.get('unmapped_aliase').get(idx).get("doc_count"))
    size.append(api_response.get('unmapped_aliase').get(idx).get("store.size"))
    

In [46]:
df_unmapped_dict = {
        "environment" : es_client,
        "unmapped_index_name" : list(api_response['unmapped_aliase'].keys()),
        "the number of docs" : docs,
        "store.size" : size
    }

In [47]:
df = pd.DataFrame.from_dict(df_unmapped_dict)
# df.head(10)
display(df)

,environment,unmapped_index_name,the number of docs,store.size
0,"<Elasticsearch([{'host': 'localhost', 'port': ...",my-index-01,1,4.2kb
1,"<Elasticsearch([{'host': 'localhost', 'port': ...",test-index,1,5.3kb
2,"<Elasticsearch([{'host': 'localhost', 'port': ...",test-index1,0,955b


In [48]:
# *** Validate the total count of each index between source/dest ES cluster **** 
# import pytest
import ipytest

In [49]:
''' ******* Test code ********* '''
@pytest.mark.skip(reason="no way of currently testing this")
def test_pytest_skip_func():
    assert 42 == 42

In [50]:
def test_pytest_func():
    assert 42 == 42

In [51]:
def test_index_analyze():
    response = es_client.indices.analyze(index="test-index", body={
            "analyzer": "standard",
            "text": "The quick",
        })
    assert response == {
      "tokens": [
        {
          "token": "the",
          "start_offset": 0,
          "end_offset": 3,
          "type": "<ALPHANUM>",
          "position": 0
        },
        {
          "token": "quick",
          "start_offset": 4,
          "end_offset": 9,
          "type": "<ALPHANUM>",
          "position": 1
        }
      ]
    }

In [52]:
''' Execute the tests using ipytest.run(). You can pass command-line arguments to control test behavior: '''
''' The pytest framework makes it easy to write small, readable tests, and can scale to support complex functional testing for applications and libraries. '''
ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.11.7, pytest-8.2.2, pluggy-1.5.0 -- C:\Users\euiyoung.hwang\Git_Workspace\ELK_Stack_Upgrade\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\euiyoung.hwang\Git_Workspace\ELK_Stack_Upgrade
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.0.0
collected 3 items. 

SKIPPED [ 33%]2b.py::test_pytest_skip_func <- ..\..\AppData\Local\Temp\ipykernel_11208\1954050930.py 
PASSED [ 66%]62b.py::test_pytest_func <- ..\..\AppData\Local\Temp\ipykernel_11208\2292377223.py 
PASSED [100%]62b.py::test_index_analyze <- ..\..\AppData\Local\Temp\ipykernel_11208\3522919548.py 

======================================== warnings summary =========================================
jupyter-workflow/t_7b67379c9f0d4d179577b45b45c3262b.py::test_index_analyze
  C:\Users\euiyoung.hwang\Git_Workspace\ELK_Stack_Upgrade\.venv\Lib\site-packages\elasticsearch\connection\http_u

<ExitCode.OK: 0>